In [1]:
# Core 
import datetime
import os
import glob
from IPython.display import HTML, Image, display
import tempfile
import shutil

# Analysis 
import xarray as xr
import numpy as np
import pyproj as pp
import scipy as sp
import metpy as mp
import networkx 
from networkx.algorithms.components.connected import connected_components

# Plotting 
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib.ticker as mticker
from matplotlib.animation import FuncAnimation
import matplotlib.colors as colors

# Debugging 
import pdb
%load_ext line_profiler

# Radar Tools
import pyart
import tint
from tint.data_utils import get_nexrad_keys, read_nexrad_key
from tint import Cell_tracks, animate
from tint.visualization import embed_mp4_as_gif


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm.py:104: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:
/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.04/lib/python3.6/site-packages/pyart/graph/cm_colorblind.py:32: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'red' in spec:


In [67]:
# Store filenames for an example day of data
filenames = sorted(glob.glob(('/g/data/rr5/CPOL_radar/CPOL_level_1b/'
                              + 'GRIDDED/GRID_70km_1000m/'
                              + '2016/20161220/*.nc')))
# Generate grid generator 
# Note generators produce iterators
# These are alternative to using lists and looping
grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') 
         for fn in filenames)

# Define settings for tracking
settings = {
    'MIN_SIZE' : 30, 
    'FIELD_THRESH' : 20,
    'GS_ALT' : 1500,
    'LEVELS' : np.array(
        [[0, 3000], 
         [3000, 6000]]
    )
}

# Calculate high and low level tracks
tracks_obj  = Cell_tracks()

for parameter in ['MIN_SIZE', 'FIELD_THRESH', 'GS_ALT', 'LEVELS']:
    tracks_obj.params[parameter] = settings[parameter]

# Calculate tracks
# tracks_obj.get_tracks(grids)

In [58]:
upper = np.array([
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 1, 1, 1, 0, 2, 2, 2],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 3, 0, 0, 0, 4, 4],
    [0, 3, 3, 0, 0, 0, 4, 4],
])

mid = np.array([
    [0, 1, 1, 1, 1, 1, 1, 4],
    [0, 1, 1, 1, 1, 1, 1, 4],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 3, 3, 3, 0, 2, 2, 2],
    [0, 3, 3, 3, 0, 2, 2, 2],
    [0, 3, 3, 3, 0, 2, 2, 2],
])

lower = np.array([
    [0, 1, 0, 2, 0, 0, 13,  4],
    [0, 1, 0, 2, 0, 3, 13,  4],
    [0, 0, 0, 0, 0, 0, 13,  0],
    [0, 5, 6, 7, 8, 9, 13, 11],
    [0, 0, 0, 0, 0, 0, 13,  0],
    [0, 15, 0, 14, 0, 0, 13, 0],
    [0, 15, 0, 14, 0, 0, 13, 12],
    [0, 15, 0, 14, 0, 0, 13, 12],
])

frames = np.array([lower, mid, upper])
f_maxes = frames.max(axis=(1,2))
# Relabel so that object numbers are unique across frames
frames_rl[:] = frames[:]
for i in range(1,frames_rl.shape[0]):
    frames_rl[i,frames_rl[i]>0] += np.cumsum(f_maxes[:-1])[i-1]  

In [59]:
frames_rl

array([[[ 0,  1,  0,  2,  0,  0, 13,  4],
        [ 0,  1,  0,  2,  0,  3, 13,  4],
        [ 0,  0,  0,  0,  0,  0, 13,  0],
        [ 0,  5,  6,  7,  8,  9, 13, 11],
        [ 0,  0,  0,  0,  0,  0, 13,  0],
        [ 0, 15,  0, 14,  0,  0, 13,  0],
        [ 0, 15,  0, 14,  0,  0, 13, 12],
        [ 0, 15,  0, 14,  0,  0, 13, 12]],

       [[ 0, 16, 16, 16, 16, 16, 16, 19],
        [ 0, 16, 16, 16, 16, 16, 16, 19],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 18, 18, 18,  0, 17, 17, 17],
        [ 0, 18, 18, 18,  0, 17, 17, 17],
        [ 0, 18, 18, 18,  0, 17, 17, 17]],

       [[ 0, 20, 20, 20,  0, 21, 21, 21],
        [ 0, 20, 20, 20,  0, 21, 21, 21],
        [ 0, 20, 20, 20,  0, 21, 21, 21],
        [ 0, 20, 20, 20,  0, 21, 21, 21],
        [ 0, 20, 20, 20,  0, 21, 21, 21],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 22, 22,  0,  0,  0, 23, 23],
        [ 0, 22, 22,  0,  0,  

In [60]:
# Create graph for cells that overlap at different vertical levels.
total_objs = frames_rl[-1].max()

overlap_graph = networkx.Graph()
overlap_graph.add_nodes_from(set(range(1, total_objs)))

# Create edges between the objects that overlap vertically.
for i in range(frames_rl.shape[0]-1):
    # For each object in frame i, work out the objects it overlaps in 
    # the frame below.
    
    # Determine the objects in frame i.
    objects = set(frames_rl[i][frames_rl[i]>0])
    # Determine the objects in frame i+1.
    objects_next = set(frames_rl[i+1][frames_rl[i+1]>0])
    for j in range(len(list(objects))):
        overlap = np.logical_and(frames_rl[i] == list(objects)[j], 
                                 frames_rl[i+1] > 0)
        overlap_objs = set((frames_rl[i+1][overlap]).flatten())
        # If objects overlap, add edge between object j and first 
        # object from overlap set
        if bool(overlap_objs):
            overlap_graph.add_edges_from(
                [(list(objects)[j], list(overlap_objs)[0])]
            )
            # Add edges between objects in overlap set
            for k in range(0, len(list(overlap_objs))-1):
                overlap_graph.add_edges_from(
                    [(list(overlap_objs)[k], list(overlap_objs)[k+1])]
                )

In [62]:
# Create frames_sys based on connected components 
new_objs = list(connected_components(overlap_graph))
frames_sys = np.zeros(frames_rl.shape)
for i in range(len(new_objs)):
    frames_sys[np.isin(frames_rl, list(new_objs[i]))] = i + 1

In [64]:
frames_sys

array([[[0., 1., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 2., 3., 4., 5., 6., 1., 8.],
        [0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 9., 0., 9., 0., 0., 1., 0.],
        [0., 9., 0., 9., 0., 0., 1., 1.],
        [0., 9., 0., 9., 0., 0., 1., 1.]],

       [[0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 9., 9., 9., 0., 1., 1., 1.],
        [0., 9., 9., 9., 0., 1., 1., 1.],
        [0., 9., 9., 9., 0., 1., 1., 1.]],

       [[0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 1., 1., 1., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 9., 9., 0., 0., 0., 1., 1.],
        [0., 9., 9., 0., 0., 0

In [ ]:
# Specify rounded bounds of CPOL 1000 m gridded dataset 
lon_min = 130.40
lon_max = 131.69

lat_max = -11.61
lat_min = -12.87

grids = (pyart.io.read_grid(fn, include_fields = 'reflectivity') for fn in filenames)

tmp_dir = tempfile.mkdtemp()

# Animate low level tracks
animate(tracks_obj, grids, os.path.join(tmp_dir, 'tracks_anim'),
        lat_lines=np.arange(lat_min, lat_max, .1),
        lon_lines=np.arange(lon_min, lon_max, .1),
        tracers=True)

embed_mp4_as_gif(os.path.join(tmp_dir, 'tracks_anim.mp4'))

shutil.rmtree(tmp_dir)

In [ ]:
tracks_obj.tracks.groupby(level='uid').size().sort_values(ascending=False)[:10]

In [ ]:
# Lagrangian Animation
tmp_dir = tempfile.mkdtemp()

grids = (pyart.io.read_grid(fn) for fn in filenames)  # refresh grid generator
animate(tracks_obj, grids, os.path.join(tmp_dir, 'lagrangian'), style='lagrangian', uid='151', alt=1500)

embed_mp4_as_gif(os.path.join(tmp_dir, 'lagrangian.mp4'))
shutil.rmtree(tmp_dir)

In [68]:
grid_list = list(grids)

In [70]:
grid_list[0].nx

141